## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-30-06-58-21 +0000,bbc,Olympic star 'seriously' hurt in climbing acci...,https://www.bbc.com/sport/articles/c4g627p4q0k...
1,2025-07-30-06-58-01 +0000,nyt,What is the Pacific ‘Ring of Fire’?,https://www.nytimes.com/article/ring-of-fire-e...
2,2025-07-30-06-56-46 +0000,bbc,Lammy hopes plan to recognise Palestinian stat...,https://www.bbc.com/news/articles/c23p8gl05r2o...
3,2025-07-30-06-56-11 +0000,nyt,Here’s the latest on the tsunami.,https://www.nytimes.com/live/2025/07/29/world/...
4,2025-07-30-06-50-51 +0000,nypost,Hofstra baseball coach reveling in St. James-S...,https://nypost.com/2025/07/30/sports/hofstra-b...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2299/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
99,trump,64
199,new,20
174,gaza,20
13,palestinian,18
18,tsunami,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
124,2025-07-29-22-00-59 +0000,nyt,"Trump Says Gaza Will Get More Aid, but Israel ...",https://www.nytimes.com/2025/07/29/us/politics...,127
53,2025-07-30-01-39-32 +0000,nypost,Senate confirms Trump lawyer Emil Bove for fed...,https://nypost.com/2025/07/29/us-news/senate-c...,123
328,2025-07-29-09-30-00 +0000,wsj,Capital Account: Trump has hit his tariff goal...,https://www.wsj.com/economy/trade/trumps-new-t...,120
216,2025-07-29-17-49-06 +0000,nypost,"Moscow taunts Trump by comparing him to Biden,...",https://nypost.com/2025/07/29/world-news/mosco...,119
55,2025-07-30-01-30-00 +0000,wsj,The Senate confirmed former Trump defense atto...,https://www.wsj.com/politics/policy/trumps-for...,116


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
124,127,2025-07-29-22-00-59 +0000,nyt,"Trump Says Gaza Will Get More Aid, but Israel ...",https://www.nytimes.com/2025/07/29/us/politics...
259,78,2025-07-29-15-30-51 +0000,nypost,Union Pacific strikes $85B deal to buy rival N...,https://nypost.com/2025/07/29/business/union-p...
51,71,2025-07-30-01-49-26 +0000,nypost,Reno casino victim ID’d as Calif. dad who advo...,https://nypost.com/2025/07/29/us-news/reno-cas...
53,59,2025-07-30-01-39-32 +0000,nypost,Senate confirms Trump lawyer Emil Bove for fed...,https://nypost.com/2025/07/29/us-news/senate-c...
180,57,2025-07-29-19-52-56 +0000,nyt,UK Will Recognize Palestinian Statehood In Sep...,https://www.nytimes.com/2025/07/29/world/europ...
84,55,2025-07-30-00-06-03 +0000,nypost,NYC skyscraper that turned into tower of terro...,https://nypost.com/2025/07/29/us-news/nyc-skys...
133,51,2025-07-29-21-42-41 +0000,nypost,Top Zelensky official praises Trump for giving...,https://nypost.com/2025/07/29/us-news/top-zele...
329,46,2025-07-29-09-24-00 +0000,wsj,Europe’s Star Economy Picks Up Pace as Fresh T...,https://www.wsj.com/world/europe/europes-star-...
68,46,2025-07-30-00-49-48 +0000,latimes,California coast under tsunami alert; North Co...,https://www.latimes.com/california/story/2025-...
49,44,2025-07-30-01-52-23 +0000,wapo,Trump softens stance on China amid trade talks...,https://www.washingtonpost.com/national-securi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
